<a href="https://colab.research.google.com/github/Maganti1205/earthengine-api/blob/master/EarthEngineDemo_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo Scenario

 Extreme weather events have a devastating impact around the world. Flooding, heat waves, and drought have substantial human and financial costs, causing mortality and devastation of homes and property. The following example shows how to use satellite data mosaics from Earth Engine and open road datasets from BigQuery, processing the data in both environments to determine which road segments are affected by a flooding event in the UK.


 ## Demo Flow

 Earth Engine => Big Query => Visualization

 ## Prerequisites


1.   Install the dependencies {--geemap , earthengine-api}
2.   Create a new GCP project and check that billing is enabled. Alternatively an existing project with valid billing ID could be used.
3.   Enable the BigQuery and Earth Engine APIs.
4.   Make sure to have the permissions to create dataset.
5.   Make sure to have the below IAM roles assigned on the Bigquery dataset

>1. bigquery.dataEditor + bigquery.jobUser

>2. bigquery.dataOwner + bigquery.jobUser

>3. bigquery.admin




























In [1]:
# @title Prerequisites
# Install 'geemap' libary to display the map
!pip install geemap
!pip install earthengine-api --upgrade

In [ ]:
# @title Parameter Setup
project_id = "my-demo-project-360918"  # @param {type:"string"} #replace the project id with your project
dataset_id = "ee_export"
table_id = "ee_test"
table =  dataset_id + "." + table_id
region = 'us'
table_path = project_id + "." + dataset_id + "." + table_id
print("Region:      ",region)
print("Table Path:  ",table_path)


In [ ]:
# @title Authenticate and initialize the Session
import google
import ee
from google.cloud import bigquery
from google.colab import auth as google_auth
client= bigquery.Client()
google_auth.authenticate_user()
credentials, auth_project_id = google.auth.default()
ee.Initialize(credentials, project=project_id)

In [ ]:
# @title Create BQ Dataset
# Create BQ Project
!bq --location={region} mk --dataset {project_id}:{dataset_id}

Define area of interest and centre point

In [ ]:
# @title Define point and Area of Interest(aoi)
# Define Point
pt = ee.Geometry.Point([-2.788305016918912,53.99351387096434])

# Define Polygons
aoi =ee.Geometry.Polygon([[-2.918327782772696,53.93320265050595],
                          [-2.4572453242766024,53.93320265050595],
                          [-2.4572453242766024,54.07928378526107],
                          [-2.918327782772696,54.07928378526107],
                          [-2.918327782772696,53.93320265050595]
                           ])


In [ ]:
# @title Display aoi and point
import geemap
Map = geemap.Map()
Map.centerObject(pt, 10);
Map.setOptions(mapTypeId='HYBRID', styles={}, types=[])
Map.addLayer(aoi,{"color":"blue"});
Map.addLayer(pt,{"color":"red"});
Map

The Earth Engine Data Catalog contains the [Copernicus Sentinel Synthetic Aperture Radar collection](https://colab.research.google.com/drive/11Cr9nqizvw4D-SwSKgV2mZ6njFiy1lac#scrollTo=ycom7Zl36R4I&line=1&uniqifier=1). This public dataset is composed of radar images that measure how surfaces scatter light waves back to a satellite's sensor. Standing bodies of water act like mirrors for radio signals, reflecting the satellite's radar light away rather than scattering it back to the imaging sensor. Most natural surfaces don't have this property, which means that one can differentiate standing bodies of water from their surroundings by looking for "dark" patches in the images (that is, areas with low backscatter values). Let’s prepare the input data by selecting an area of interest and filtering images with vertical-vertical ("VV") polarization, sending vertically polarized light, and measuring the vertically polarized light that's returned.

In [ ]:
# @title Data Collections
# Load Sentinel-1 C-band SAR Ground Range collection (log scaling, VV co-polar)
collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(pt).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).select('VV')
# Filter by date
before = collection.filterDate('2017-11-01', '2017-11-17').mosaic() #before floods
after = collection.filterDate('2017-11-18', '2017-11-23').mosaic() #after floods

In [ ]:
# @title Identify Flooded Areas
# Threshold smoothed radar intensities to identify "flooded" areas.
smoothing_radius = 100
diff_upper_threshold = -3
diff_smoothed = after.focal_median(smoothing_radius, 'circle', 'meters').subtract(before.focal_median(smoothing_radius, 'circle', 'meters'));
diff_thresholded = diff_smoothed.lt(diff_upper_threshold)

In [ ]:
# @title Remove Water Areas(other than floods)
# Remove global surface water, ie, oceans, lakes, etc
waterOcc = ee.Image("JRC/GSW1_0/GlobalSurfaceWater").select('occurrence')
jrc_data0 = ee.Image("JRC/GSW1_0/Metadata").select('total_obs').lte(0)
waterOccFilled = waterOcc.unmask(0).max(jrc_data0)
waterMask = waterOccFilled.lt(10)
diff_thresholded = diff_thresholded.updateMask(waterMask)

In [ ]:
# @title Visualize the Map with Flooded Area
# Display flooded areas on the map
import geemap
vis_params = {
    "palette": ["0000FF"],
}
Map = geemap.Map()
Map.setOptions(mapTypeId='HYBRID', styles={}, types=[])
Map.centerObject(pt, 10);
Map.addLayer(diff_thresholded.updateMask(diff_thresholded),vis_params,'flooded areas - blue', 1)
Map

New connector supports export of vector data. Let’s convert flooded pixel data to vector format.

In [ ]:
#@title Extract Vectors from the Flooded areas
#Extract vectors from the diff threshold to load to Bigquery
vectors = diff_thresholded.reduceToVectors(geometry = aoi, scale = 10,geometryType = 'polygon',eightConnected = True)

This is where our new BigQuery connector simplifies export to just making one call: Export.table.toBigQuery.

In [ ]:
#@title Export to BigQuery
task_config = { 'collection': vectors,
  'description':'ee2bq_export_polygons',
  'table': table_path,
  'overwrite': True
}
task = ee.batch.Export.table.toBigQuery(**task_config)
task.start() # after a few minutes it will be completed, check task.status() to see the result


In [ ]:
#@title Check Export Job Status
# Check the results and make sure the status is COMPLETED before checking the results in BigQuery
task.status()

Now we have exported data available in BigQuery. Execute the query to check the data

In [ ]:
#@title Check Bigquery Results
#%%substitute_globals
%%bigquery --project $project_id
SELECT * from ee_export.ee_test #replace it with your dataset and tablename

#If you get a "table not found" error then check the step above to see if your job has completed

In our example we are going to use the public “planet_ways” dataset from OpenStreetMap, so we can find roads that are underwater.


> Get polygons corresponding to flat areas from the exported table.

> Filter out administrative areas

> Join result with road polygons from OpenStreetMap data that intersect with flat areas.


In [ ]:
#@title BQ Result set to display flooded highways
%%bigquery regions_by_country --project $project_id
SELECT
 id, area,version,changeset,osm_timestamp,ST_ASGEOJSON(flood_poly) as flood_poly,
 ST_ASGEOJSON(road_geometry) as road_geometry
FROM (
 -- query 1 - find all the flooding areas
 SELECT
   geo AS flood_poly,
   ST_AREA(geo) AS area
 FROM
   ee_export.ee_test #replace it with your dataset and tablename
 WHERE
   ST_AREA(geo) < 500000 ) t1 -- eliminate admin areas in the dataset
JOIN (
 -- query 2 - find all the highways in Open Street Map - https://wiki.openstreetmap.org/wiki/BigQuery_dataset#Query_2:_hospitals_with_no_phone_tag
 SELECT
   id,
   version,
   changeset,
   osm_timestamp,
   geometry as road_geometry
 FROM
   `bigquery-public-data.geo_openstreetmap.planet_ways` planet_ways,
   planet_ways.all_tags AS all_tags
 WHERE
 -- this tag catches all types of roads https://wiki.openstreetmap.org/wiki/Map_features
   all_tags.key = 'highway' )
ON
 ST_INTERSECTS(flood_poly, road_geometry)

In [ ]:
#@title Extract the Features from flood_poly
floods = '{"type": "FeatureCollection", "features":['
floods += regions_by_country.flood_poly.str.cat(sep=", ")
floods += ']}'

In [ ]:
#@title Extract the features from road_geometry
highways = '{"type": "FeatureCollection", "features":['
highways += regions_by_country.road_geometry.str.cat(sep=", ")
highways += ']}'


In [ ]:
#@title Display Flooded Areas and  Highways using geemap
import geemap
from ipyleaflet import GeoJSON
import json

styling = {"color":"red","fillcolor":"red"}

Flooded_Highways = GeoJSON(
    data=json.loads(floods),
    name='Flooded areas'
)

Flooded_Areas = GeoJSON(
    data=json.loads(highways),
    name='Flooded roads',
    style = styling
)
Map = geemap.Map()
Map.setOptions(mapTypeId='HYBRID', styles={}, types=[])
Map.centerObject(pt, 12)
Map.add_layer(Flooded_Highways)
Map.add_layer(Flooded_Areas)
Map